# QDP: GPU-Accelerated Quantum Data Preparation

Welcome to **QDP** (Quantum Data Preparation). This library accelerates the encoding of classical data into quantum states using GPUs.

**In this tutorial, you will learn:**
1. How to install QDP on Google Colab.
2. How to initialize the QDP Engine.
3. How to encode data (Lists, NumPy, PyTorch) into quantum states.
4. How to integrate with PyTorch for quantum-classical workflows.

## 1. Environment Setup

First, we verify we have a GPU, install Rust (needed to compile the extension), and install the QDP package.

In [ ]:
# Check for NVIDIA GPU
!nvidia-smi

In [ ]:
# 1. Install Rust Toolchain (Required for compilation)
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
import os
os.environ['PATH'] += ":/root/.cargo/bin"

# 2. Clone Mahout Repository
!git clone https://github.com/apache/mahout.git

# 3. Install QDP Package
%cd /content/mahout/qdp/qdp-python
!pip install .
!pip install torch numpy

## 2. Basic Usage

Now we can initialize the engine and perform our first encoding.

In [ ]:
import torch
import numpy as np
# QDP is currently exposed via the `_qdp` extension module or `qumat.qdp` package depending on install.
# Adjust based on final package structure.
try:
    from qumat_qdp import QdpEngine
    print("Imported QdpEngine from qumat_qdp")
except ImportError:
    try:
        from _qdp import QdpEngine
        print("Imported QdpEngine from _qdp")
    except ImportError:
         print("Could not import QdpEngine. Did installation succeed?")

# Initialize Engine on GPU 0
try:
    engine = QdpEngine(0)
    print("QDP Engine Initialized successfully on GPU 0")
except Exception as e:
    print(f"Failed to initialize engine: {e}")

In [ ]:
# Example 1: Encode a simple Python List
data = [0.5, 0.5, 0.5, 0.5]
n_qubits = 2

# Encode using Amplitude Encoding
# 4 values can form a state of 2 qubits (2^2 = 4)
qtensor = engine.encode(data, n_qubits, "amplitude")

# Convert to PyTorch Tensor (Zero-Copy)
torch_tensor = torch.from_dlpack(qtensor)

print(f"Quantum State Shape: {torch_tensor.shape}")
print(f"Quantum State Data:\n{torch_tensor}")

## 3. High-Performance Batch Processing

QDP shines when processing large batches of data. It performs normalization and state preparation entirely on the GPU.

In [ ]:
# Generate random batch of data: 100 samples, 4 features (2 qubits)
batch_size = 100
features = 4  # 2^2
input_data = np.random.rand(batch_size, features).astype(np.float32)

print(f"Encoding {batch_size} vectors...")

# Encode Batch
qtensor_batch = engine.encode(input_data, 2, "amplitude")
torch_batch = torch.from_dlpack(qtensor_batch)

print(f"Batch Shape: {torch_batch.shape}")
# Verify on GPU
print(f"Device: {torch_batch.device}")